In [11]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [21]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")


llm=ChatGroq(groq_api_key=groq_api_key,model="qwen/qwen3-32b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F0BC05A110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F0BC05BEE0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
## Hugging face embeddings

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\USER\OneDrive\Desktop\groq_LPU\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents, embeddings, collection_name="pets")
vectorstore

In [15]:
vectorstore.similarity_search("Which pets are mammals?")

[Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b2b35af6-6b45-4d53-bc45-e589c7b977cd', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e29c8482-b273-430b-aff4-18adf96e6f20', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [16]:
## Async Query
await vectorstore.asimilarity_search("Which pets are mammals?")

[Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='b2b35af6-6b45-4d53-bc45-e589c7b977cd', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='e29c8482-b273-430b-aff4-18adf96e6f20', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [17]:
vectorstore.similarity_search_with_relevance_scores("Which pets are mammals?")

C:\Users\USER\AppData\Local\Temp\ipykernel_10964\2063535583.py:1: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'), 0.14597217089017467), (Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'), 0.12427640626961578), (Document(id='b2b35af6-6b45-4d53-bc45-e589c7b977cd', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'), 0.03086938048801413), (Document(id='e29c8482-b273-430b-aff4-18adf96e6f20', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'), -0.012557804986943921)]
  vectorstore.similarity_search_with_relevance_scores("Which pets are mamm

[(Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.14597217089017467),
 (Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.12427640626961578),
 (Document(id='b2b35af6-6b45-4d53-bc45-e589c7b977cd', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.03086938048801413),
 (Document(id='e29c8482-b273-430b-aff4-18adf96e6f20', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  -0.012557804986943921)]

In [18]:
## RETRIEVE-THEN-READ
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda


retriver=RunnableLambda(vectorstore.similarity_search).bind(k=1)

retriver.batch(["cat","dog"])


[[Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)
retriver.batch(["cat","dog"])

[[Document(id='bae83edc-8798-44e6-b128-e6a57c3305a4', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='ad9855f3-4464-4221-b473-36cf9077fcd1', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [22]:
## RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)

<think>
Okay, so the user is asking about dogs. Let me check the provided context. There are two documents here. The first one with ID 'ad9855f3...' mentions that dogs are great companions known for their loyalty and friendliness. The second document talks about cats being independent, which isn't directly relevant here.

The user wants information about dogs, so I should focus on the first document. The answer needs to be based solely on the context provided. I can't use any outside knowledge. The key points from the context are that dogs are companions, loyal, and friendly. I should present that clearly. Also, make sure not to mention anything about cats since the question is about dogs. Keep the answer concise and stick to the given information. Let me structure the response to highlight their loyalty and friendliness as their main traits. That should cover it based on the provided context.
</think>

Dogs are great companions, known for their loyalty and friendliness. This makes the